In [ ]:
pip install  pandas seaborn matplotlib fuzzywuzzy python-Levenshtein


In [13]:
import pandas as pd
from fuzzywuzzy import fuzz,process
import matplotlib.pyplot as plt
import seaborn as sns


In [14]:
Titanic = pd.read_csv('Titanic.csv')

In [15]:
Titanic.head()

PassengerId  Survived  Pclass  \
0          892         0       3   
1          893         1       3   
2          894         0       2   
3          895         0       3   
4          896         1       3   

                                           Name     Sex   Age  SibSp  Parch  \
0                              Kelly, Mr. James    male  34.5      0      0   
1              Wilkes, Mrs. James (Ellen Needs)  female  47.0      1      0   
2                     Myles, Mr. Thomas Francis    male  62.0      0      0   
3                              Wirz, Mr. Albert    male  27.0      0      0   
4  Hirvonen, Mrs. Alexander (Helga E Lindqvist)  female  22.0      1      1   

    Ticket     Fare Cabin Embarked  
0   330911   7.8292   NaN        Q  
1   363272   7.0000   NaN        S  
2   240276   9.6875   NaN        Q  
3   315154   8.6625   NaN        S  
4  3101298  12.2875   NaN        S

In [16]:
Titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Survived     418 non-null    int64  
 2   Pclass       418 non-null    int64  
 3   Name         418 non-null    object 
 4   Sex          418 non-null    object 
 5   Age          332 non-null    float64
 6   SibSp        418 non-null    int64  
 7   Parch        418 non-null    int64  
 8   Ticket       418 non-null    object 
 9   Fare         417 non-null    float64
 10  Cabin        91 non-null     object 
 11  Embarked     418 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 39.3+ KB


In [17]:
Titanic.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [18]:
# Function For Fuzzy String Matching
def find_duplicates(data, column, threshold=78):
    duplicates = []
    checked = set()

    for i, name1 in enumerate(data[column]):
        if name1 in checked:
            continue
        matches = process.extract(name1, data[column], scorer=fuzz.token_sort_ratio)
        for match in matches:
            name2, score = match[0], match[1]  
            if name1 != name2 and score >= threshold:
                duplicates.append((name1, name2, score))
                checked.add(name2)
    return pd.DataFrame(duplicates, columns=["Name1", "Name2", "Similarity"])

In [19]:
# Implementation Of Function
potential_duplicates = find_duplicates(Titanic, "Name")
potential_duplicates
# -->All Names Are Found To Be Different So Will Not Drop Any Name


Name1                         Name2  Similarity
0   Myles, Mr. Thomas Francis  McCaffry, Mr. Thomas Francis          78
1     Davies, Mr. John Samuel   Andersson, Mr. Johan Samuel          78
2     Brady, Mr. John Bertram     Crafton, Mr. John Bertram          82
3   Davison, Mr. Thomas Henry      Conlon, Mr. Thomas Henry          80
4           Kiernan, Mr. John             Kennedy, Mr. John          80
5           Kiernan, Mr. John             Lingane, Mr. John          80
6           Lane, Mr. Patrick          Canavan, Mr. Patrick          79
7           Lane, Mr. Patrick          Colbert, Mr. Patrick          79
8        Angle, Mr. William A          Gilbert, Mr. William          78
9     Wiklund, Mr. Karl Johan      Salander, Mr. Karl Johan          79
10        Dennis, Mr. William           Dibden, Mr. William          82

In [20]:
# -->But If We Want To Delete We Can Do As Follows
Titanic_cleaned = Titanic[~Titanic['Name'].isin(potential_duplicates['Name1'])]
Titanic_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 409 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  409 non-null    int64  
 1   Survived     409 non-null    int64  
 2   Pclass       409 non-null    int64  
 3   Name         409 non-null    object 
 4   Sex          409 non-null    object 
 5   Age          326 non-null    float64
 6   SibSp        409 non-null    int64  
 7   Parch        409 non-null    int64  
 8   Ticket       409 non-null    object 
 9   Fare         408 non-null    float64
 10  Cabin        90 non-null     object 
 11  Embarked     409 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 41.5+ KB


In [ ]:
# Basic Data Cleaning 
Titanic['Age'].fillna(Titanic['Age'].median(), inplace=True)
Titanic['Fare'].fillna(Titanic['Fare'].mean(), inplace=True)
Titanic['Cabin'] = Titanic['Cabin'].str[0].fillna('Unknown')
Titanic['Embarked'].fillna(Titanic['Embarked'].mode()[0], inplace=True)

In [29]:
Titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Survived     418 non-null    int64  
 2   Pclass       418 non-null    int64  
 3   Name         418 non-null    object 
 4   Sex          418 non-null    object 
 5   Age          418 non-null    float64
 6   SibSp        418 non-null    int64  
 7   Parch        418 non-null    int64  
 8   Ticket       418 non-null    object 
 9   Fare         418 non-null    float64
 10  Cabin        418 non-null    object 
 11  Embarked     418 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 39.3+ KB


In [30]:
Titanic.isna().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64